# TensorFlow Profiler: Profile model performance

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [1]:
# !pip3 install -U tensorboard_plugin_profile

In [4]:
from datetime import datetime
from packaging import version
import os

import tensorflow as tf
from tensorflow import keras
tf.debugging.experimental.enable_dump_debug_info('./logs/', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
from keras import backend as K
import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

2023-09-06 19:53:29.162933: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-06 19:53:29.198259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-06 19:53:29.198973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 19:53:30.018284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: ./logs/, tensor debug mode: FULL_HEALTH)
TensorFlow version:  2.12.1


In [5]:
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2023-09-06 19:53:33.488526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


SystemError: GPU device not found

In [ ]:
# !pip3 install -U tensorflow_datasets

In [6]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [7]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)

ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [9]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])

2023-09-06 19:53:40.440554: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-09-06 19:53:40.440577: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


2023-09-06 19:53:44.223608: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2023-09-06 19:53:44.339213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-09-06 19:53:44.339575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]


Epoch 1/2
467/469 [============================>.] - ETA: 0s - loss: 0.3672 - accuracy: 0.8968

2023-09-06 19:53:59.191027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-09-06 19:53:59.191401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]


469/469 [==============================] - 19s 31ms/step - loss: 0.3664 - accuracy: 0.8971 - val_loss: 0.1925 - val_accuracy: 0.9440
Epoch 2/2
 37/469 [=>............................] - ETA: 8s - loss: 0.1919 - accuracy: 0.9453

2023-09-06 19:54:03.736229: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-09-06 19:54:03.736247: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


 57/469 [==>...........................] - ETA: 8s - loss: 0.1950 - accuracy: 0.9467

2023-09-06 19:54:04.170277: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2023-09-06 19:54:04.215774: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


469/469 [==============================] - 13s 27ms/step - loss: 0.1640 - accuracy: 0.9535 - val_loss: 0.1349 - val_accuracy: 0.9610


In [10]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

https://www.tensorflow.org/tensorboard/tensorboard_profiling_keras